In [5]:
from robinhood_auth_login import login

# Perform login
login_response = login(
    username="nandarohan442@gmail.com",
    password="Ron1506313015$",
    by_sms=True,
    store_session=False
)

# Save token to file
with open("robinhood_token.txt", "w") as f:
    f.write(login_response["access_token"])

print("✅ Token saved to robinhood_token.txt")


Verification workflow required. Please check your Robinhood app for instructions.
Waiting for challenge to be validated
5.067613840103149
Waiting for challenge to be validated
10.154067039489746
Waiting for challenge to be validated
15.225662469863892
✅ Token saved to robinhood_token.txt


In [6]:
import requests
import pandas as pd
from datetime import datetime
import os  

# Load token from file
def load_token_from_file(file_path="robinhood_token.txt"):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return f.read().strip()
    return None

# Fetch and save stock data
def fetch_and_save_stock_data(token, output_file="my_stock_data.xlsx"):
    url = "https://api.robinhood.com/positions/"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Error fetching stock data:", response.text)
        return

    positions = response.json().get("results", [])
    stock_data = []

    # Get the current timestamp
    fetch_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for position in positions:
        if float(position["quantity"]) > 0:  # Only fetch open positions
            instrument_url = position["instrument"]
            instrument_data = requests.get(instrument_url, headers=headers).json()

            symbol = instrument_data["symbol"]
            quantity = float(position["quantity"])
            average_buy_price = float(position["average_buy_price"])
            quote_url = f"https://api.robinhood.com/quotes/{symbol}/"
            current_price = float(requests.get(quote_url, headers=headers).json()["last_trade_price"])

            market_value = current_price * quantity
            profit_loss = (current_price - average_buy_price) * quantity

            stock_data.append({
                "Timestamp": fetch_time,  # Add the timestamp
                "Symbol": symbol,
                "Quantity": quantity,
                "Average Buy Price": average_buy_price,
                "Current Price": current_price,
                "Market Value": market_value,
                "Profit/Loss": profit_loss
            })

    df = pd.DataFrame(stock_data)
    df.to_excel(output_file, index=False)
    print(f"Stock data saved to {output_file}")

# Use token to fetch data
token = load_token_from_file()
if token:
    fetch_and_save_stock_data(token)
else:
    print("No token found. Please log in first.")


Stock data saved to my_stock_data.xlsx


In [7]:
import requests
import pandas as pd
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import sqlite3
import os
import tempfile

# --- CONFIG ---
RATE_LIMIT_DELAY = 1
MAX_WORKERS = 4
MIN_VOLUME = 100000
MIN_DOLLAR_VOLUME = 1_000_000
MAX_STDDEV = 0.05
MIN_PRICE = 3.0
MOMENTUM_LOOKBACK_DAYS = 5
SAVE_INTERVAL = 50
TOKEN_FILE = "robinhood_token.txt"
CHECKPOINT_FILE = "checkpoint_filtered.csv"
REJECTED_FILE = "checkpoint_rejected.csv"
DB_FILE = "filtered_tickers.db"
RETRY_FAILED_ONLY = False
FINNHUB_API_KEY = "YOUR_API_KEY"  # Replace with your actual Finnhub API key

# --- TOKEN ---
def load_token_from_file(file_path=TOKEN_FILE):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return f.read().strip()
    return None

token = load_token_from_file()
HEADERS = {"Authorization": f"Bearer {token}"} if token else {}

# --- LOGGING ---
def log(msg):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

# --- SAFE REQUEST ---
def safe_request(url, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            r = requests.get(url, headers=HEADERS)
            if r.status_code == 429:
                log(f"⏳ Rate limited. Sleeping {delay * (attempt + 1)}s...")
                time.sleep(delay * (attempt + 1))
                continue
            r.raise_for_status()
            return r
        except Exception as e:
            log(f"⚠️ Error on request to {url}: {e}")
            time.sleep(delay * (attempt + 1))
    return None

# --- HISTORICAL DATA ---
def get_history(ticker):
    url = f"https://api.robinhood.com/quotes/historicals/{ticker}/?interval=day&span=week"
    r = safe_request(url)
    if r:
        data = r.json().get("historicals", [])
        closes = [float(d["close_price"]) for d in data if d["close_price"] != "0.0000"]
        volumes = [int(d["volume"]) for d in data]
        return closes, volumes
    return [], []

# --- QUOTE CHECK ---
def is_valid_quote(ticker):
    url = f"https://api.robinhood.com/quotes/{ticker}/"
    r = safe_request(url)
    if r and r.status_code == 200:
        data = r.json()
        return data.get("last_trade_price") is not None
    return False

# --- FINNHUB FUNDAMENTALS ---
def get_sector(symbol):
    try:
        url = f"https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={FINNHUB_API_KEY}"
        r = requests.get(url)
        if r.status_code == 200:
            return r.json().get("finnhubIndustry", "Unknown")
    except:
        pass
    return "Unknown"

def get_fundamentals(symbol):
    try:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={symbol}&metric=all&token={FINNHUB_API_KEY}"
        r = requests.get(url)
        if r.status_code == 200:
            metrics = r.json().get("metric", {})
            return {
                "PE": metrics.get("peBasicExclExtraTTM", None),
                "MarketCap": metrics.get("marketCapitalization", None),
                "DebtEquity": metrics.get("totalDebt/totalEquityAnnual", None)
            }
    except:
        pass
    return {"PE": None, "MarketCap": None, "DebtEquity": None}

# --- FILTERING LOGIC ---
def check_liquidity(closes, volumes):
    if len(closes) < 3 or sum(volumes) == 0:
        return False
    avg_vol = sum(volumes) / len(volumes)
    avg_dollar_vol = sum(c * v for c, v in zip(closes, volumes)) / len(closes)
    return avg_vol >= MIN_VOLUME and avg_dollar_vol >= MIN_DOLLAR_VOLUME

def check_volatility(closes):
    returns = pd.Series(closes).pct_change().dropna()
    return returns.std() <= MAX_STDDEV

def check_price(closes):
    return closes[-1] >= MIN_PRICE

def check_momentum(closes):
    return len(closes) > MOMENTUM_LOOKBACK_DAYS and closes[-1] > closes[-MOMENTUM_LOOKBACK_DAYS]

# --- PROCESS TICKER ---
def process_ticker(item):
    ticker = item["symbol"]
    name = item.get("name", "")

    try:
        if not is_valid_quote(ticker):
            return None, {"Ticker": ticker, "Reason": "Invalid quote"}

        closes, volumes = get_history(ticker)
        if not closes or not volumes:
            return None, {"Ticker": ticker, "Reason": "No history"}

        if not check_price(closes):
            return None, {"Ticker": ticker, "Reason": "Price < $3"}

        if not check_liquidity(closes, volumes):
            return None, {"Ticker": ticker, "Reason": "Liquidity fail"}

        if not check_volatility(closes):
            return None, {"Ticker": ticker, "Reason": "Volatility fail"}

        if not check_momentum(closes):
            return None, {"Ticker": ticker, "Reason": "No uptrend"}

        sector = get_sector(ticker)
        fundamentals = get_fundamentals(ticker)

        log(f"✅ {ticker} passed all filters.")
        return {
            "Ticker": ticker,
            "Name": name,
            "Sector": sector,
            "Price": closes[-1],
            "PE": fundamentals["PE"],
            "MarketCap": fundamentals["MarketCap"],
            "DebtEquity": fundamentals["DebtEquity"]
        }, None
    except Exception as e:
        log(f"❌ Error processing {ticker}: {e}")
        return None, {"Ticker": ticker, "Reason": "Processing exception"}

# --- SAVE TO SQLITE ---
def save_to_sqlite(data, db_path=DB_FILE):
    conn = sqlite3.connect(db_path)
    df = pd.DataFrame(data)
    df.to_sql("FilteredTickers", conn, if_exists="replace", index=False)
    conn.commit()
    conn.close()
    log(f"📦 Data saved to {db_path}")

# --- CHECKPOINT ---
def save_checkpoint(data, path):
    temp_path = tempfile.mktemp()
    pd.DataFrame(data).to_csv(temp_path, index=False)
    os.replace(temp_path, path)

# --- MAIN ---
def main():
    final_filtered = pd.read_csv(CHECKPOINT_FILE).to_dict('records') if os.path.exists(CHECKPOINT_FILE) else []
    rejected_tickers = pd.read_csv(REJECTED_FILE).to_dict('records') if os.path.exists(REJECTED_FILE) else []
    already_processed = set(d['Ticker'] for d in final_filtered + rejected_tickers)

    if RETRY_FAILED_ONLY:
        tickers_to_process = [{"symbol": t["Ticker"], "name": ""} for t in rejected_tickers]
        final_filtered = []
        rejected_tickers = []
        log(f"🔁 Retrying {len(tickers_to_process)} previously rejected tickers...")
    else:
        url = "https://api.robinhood.com/instruments/"
        next_url = url
        tickers_to_process = []
        log("📥 Gathering tradable stock tickers...")

        while next_url:
            r = safe_request(next_url)
            if not r:
                break
            data = r.json()
            for item in data["results"]:
                if item["type"] == "stock" and item["tradeable"]:
                    if item["symbol"] not in already_processed:
                        tickers_to_process.append({"symbol": item["symbol"], "name": item["name"]})
            next_url = data.get("next")

    log(f"🔍 Filtering {len(tickers_to_process)} tickers...")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(process_ticker, item): item["symbol"] for item in tickers_to_process}
        for idx, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Processing")):
            ticker = futures[future]
            try:
                result, error = future.result()
                if result:
                    final_filtered.append(result)
                elif error:
                    rejected_tickers.append(error)
            except Exception as e:
                log(f"❌ Future failed for {ticker}: {e}")
                rejected_tickers.append({"Ticker": ticker, "Reason": "Unhandled exception"})

            if (len(final_filtered) + len(rejected_tickers)) % SAVE_INTERVAL == 0:
                save_checkpoint(final_filtered, CHECKPOINT_FILE)
                save_checkpoint(rejected_tickers, REJECTED_FILE)
                log(f"💾 Checkpoint saved at {len(final_filtered)} valid, {len(rejected_tickers)} rejected.")

            time.sleep(RATE_LIMIT_DELAY)

    save_to_sqlite(final_filtered)
    log(f"✅ Done. {len(final_filtered)} valid tickers, {len(rejected_tickers)} rejected.")

if __name__ == "__main__":
    main()


[19:14:10] 📥 Gathering tradable stock tickers...
[19:15:38] 🔍 Filtering 37 tickers...


Processing: 100%|██████████████████████████████████████████████████████████████████████| 37/37 [00:37<00:00,  1.02s/it]

[19:16:16] 📦 Data saved to filtered_tickers.db
[19:16:16] ✅ Done. 2569 valid tickers, 2918 rejected.
